In [7]:
import os
import re
import pandas as pd
import sys
import math
from twitter_to_sqlite import utils

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path + "/tweetfeed")

In [8]:
auth_path = "../auth/auth.json"
owner_id = "143058191"

In [9]:
from data import load_tweets, find_news, get_domain, find_url
from twitter_utils import timeout_handling, get_collection_id

# add accounts to list

In [15]:
def get_list_id(owner_id, list_name, auth_path):
    auth = json.load(open(auth_path))
    session = utils.session_for_auth(auth)
    url = f"https://api.twitter.com/1.1/lists/list.json?user_id={owner_id}"
    response = session.get(url)
    timeout_handling(response, sleep=900)
    while True:
        if response.reason == "OK":
            try:
                list_id = ""
                for item in response.json():
                    if item["name"] == list_name:
                        list_id = item["id"]
                        return list_id
                if list_id == "":
                    raise ValueError(f"ValueError: No list with '{list_name}' name")
            except ValueError:
                raise

In [17]:
get_list_id(owner_id, "muted", auth_path)

1351254441798807555

In [19]:
accounts = pd.read_csv("list.csv", names=["acc"]).acc.tolist()

In [58]:
def add_to_list(owner_id, auth_path: str, accounts_list: list, list_name: str):
    auth = json.load(open(auth_path))
    session = utils.session_for_auth(auth)
    muted_list = get_list_id(owner_id, list_name, auth_path)
    for i in accounts_list:
        try:
            url = f"https://api.twitter.com/1.1/lists/members/create.json?screen_name={i}&list_id={muted_list}"
            response = session.post(url)
            while response.reason != "OK":
                print(response.reason, response.text)
                if response.reason == "Too Many Requests":
                    print(f"Rate limit error - waiting for {sleep} seconds")
                    time.sleep(sleep)
                else:
                    break
            if "errors" in response.json():
                print(i, ": ", response.json()["errors"][0]["message"])
        except Exception as ex:
            raise(ex)
    print("Done!")

In [ ]:
# TODO - refractor with create_all? also - create new list ?

In [59]:
# add_to_list(owner_id, auth_path, accounts, "nytblock")

Done!


In [64]:
from twitter_utils import timeout_handling, get_users_from_list

In [65]:
mutedacc_rich = get_users_from_list(owner_id, auth_path, list_name="muted")

In [76]:
mutedacc = [user["id"] for user in mutedacc_rich]

In [77]:
def remove_to_list(owner_id, auth_path: str, accounts_list: list, list_name: str):
    auth = json.load(open(auth_path))
    session = utils.session_for_auth(auth)
    muted_list = get_list_id(owner_id, list_name, auth_path)
    for i in accounts_list:
        try:
            url = f"https://api.twitter.com/1.1/lists/members/destroy.json?user_id={i}&list_id={muted_list}"
            response = session.post(url)
            while response.reason != "OK":
                print(response.reason, response.text)
                if response.reason == "Too Many Requests":
                    print(f"Rate limit error - waiting for {sleep} seconds")
                    time.sleep(sleep)
                else:
                    break
            if "errors" in response.json():
                print(i, ": ", response.json()["errors"][0]["message"])
        except Exception as ex:
            raise(ex)
    print("Done!")

In [78]:
# remove_to_list(owner_id, auth_path, mutedacc, "nytblock")

Bad Request {"errors":[{"code":110,"message":"The user you are trying to remove from the list is not a member."}]}
2859448331 :  The user you are trying to remove from the list is not a member.
Bad Request {"errors":[{"code":110,"message":"The user you are trying to remove from the list is not a member."}]}
1269670536 :  The user you are trying to remove from the list is not a member.
Bad Request {"errors":[{"code":110,"message":"The user you are trying to remove from the list is not a member."}]}
369931522 :  The user you are trying to remove from the list is not a member.
Done!
